[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Fq_learning%2Facrobot_dqn.ipynb)

You can easily run this notebook at https://lab.mlpack.org/

This notebook is shows how to get use 3-Step Double DQN with Prioritized Replay to train an agent to get high scores for the [Acrobot](https://gym.openai.com/envs/Acrobot-v1) environment. 

We make the agent train and test on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/core.hpp>

In [2]:
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/q_learning.hpp>
#include <mlpack/methods/reinforcement_learning/q_networks/simple_dqn.hpp>
#include <mlpack/methods/reinforcement_learning/environment/env_type.hpp>
#include <mlpack/methods/reinforcement_learning/policy/greedy_policy.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [3]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [4]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [5]:
using namespace mlpack;

In [6]:
using namespace mlpack::ann;

In [7]:
using namespace ens;

In [8]:
using namespace mlpack::rl;

## Initializing the agent

In [9]:
// Set up the state and action space.
DiscreteActionEnv::State::dimension = 2;
DiscreteActionEnv::Action::size = 3;

In [10]:
// Set up the network.
FFN<MeanSquaredError<>, GaussianInitialization> network(
    MeanSquaredError<>(), GaussianInitialization(0, 0.001));
network.Add<Linear<>>(DiscreteActionEnv::State::dimension, 128);
network.Add<ReLULayer<>>();
network.Add<Linear<>>(128, DiscreteActionEnv::Action::size);
// Set up the network.
SimpleDQN<> model(network);

In [11]:
// Set up the policy method.
GreedyPolicy<DiscreteActionEnv> policy(1.0, 1000, 0.1, 0.99);
RandomReplay<DiscreteActionEnv> replayMethod(32, 10000);

In [12]:
// Set up training configurations.
TrainingConfig config;
config.TargetNetworkSyncInterval() = 100;
config.ExplorationSteps() = 200*10;

In [13]:
// Set up DQN agent.
QLearning<DiscreteActionEnv, decltype(model), AdamUpdate, decltype(policy), decltype(replayMethod)>
    agent(config, model, policy, replayMethod);

## Preparation for training the agent

In [14]:
// Set up the gym training environment.
gym::Environment env("gym.kurg.org", "4040", "MountainCar-v0");

// Initializing training variables.
std::vector<double> returnList;
size_t episodes = 0;
bool converged = true;

// The number of episode returns to keep track of.
size_t consecutiveEpisodes = 50;

In [15]:
// Function to train the agent on mlpack's own implementation of the CartPole environment.
void train(const size_t numSteps)
{
  agent.Deterministic() = false;
  std::cout << "Training for " << numSteps << " steps." << std::endl;
  while (agent.TotalSteps() < numSteps)
  {
    double episodeReturn = 0;
    double adjustedEpisodeReturn = 0;
    env.reset();
    do
    {
      agent.State().Data() = env.observation;
      agent.SelectAction();
      arma::mat action = {double(agent.Action().action)};

      env.step(action);
      DiscreteActionEnv::State nextState;
      nextState.Data() = env.observation;
      
      // Use an adjusted reward for task completion.
      double adjustedReward = env.reward;
      if (nextState.Data()[0] < -0.8)
        adjustedReward += 0.5;

      replayMethod.Store(agent.State(), agent.Action(), adjustedReward, nextState,
          env.done, 0.99);
      episodeReturn += env.reward;
      adjustedEpisodeReturn += adjustedReward;
      agent.TotalSteps()++;
      if (agent.Deterministic() || agent.TotalSteps() < config.ExplorationSteps())
        continue;
      agent.TrainAgent();
    } while (!env.done);
    returnList.push_back(episodeReturn);
    episodes += 1;

    if (returnList.size() > consecutiveEpisodes)
      returnList.erase(returnList.begin());
        
    double averageReturn = std::accumulate(returnList.begin(),
                                           returnList.end(), 0.0) /
                           returnList.size();
    if(episodes % 1 == 0)
    {
      std::cout << "Avg return in last " << consecutiveEpisodes
          << " episodes: " << averageReturn
          << "\t Episode return: " << episodeReturn
          << "\t Adjusted return: " << adjustedEpisodeReturn
          << "\t Total steps: " << agent.TotalSteps() << std::endl;
    }
  }
}

## Let the training begin

In [16]:
// Training the agent for a total of at least 100 episodes.
train(200*100)

Training for 20000 steps.
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 1000
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 1200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 1400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 1600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 1800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted 

Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 16400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -193.5	 Total steps: 16600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 16800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 17000
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -188.5	 Total steps: 17200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -190.5	 Total steps: 17400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 17600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -189.5	 Total steps: 17800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -171	 Total steps: 18000
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted retur

## Testing the trained agent

In [17]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "MountainCar-v0");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();

auto video = xw::video_from_url(url).finalize();
video

 Total steps: 200	 Total reward: -200


A Jupyter widget

## A little more training...

In [18]:
// Training the same agent for a total of at least 300 episodes.
train(200*300)

Training for 60000 steps.
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -192.5	 Total steps: 20200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 20400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -190	 Total steps: 20600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -186	 Total steps: 20800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 21000
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 21200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -180.5	 Total steps: 21400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -192.5	 Total steps: 21600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -175	 Total steps: 21800
Avg return in last 50 episodes: -200	 Episode retu

Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -192	 Total steps: 36200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 36400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 36600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 36800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -191	 Total steps: 37000
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -187.5	 Total steps: 37200
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -191.5	 Total steps: 37400
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -200	 Total steps: 37600
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return: -186.5	 Total steps: 37800
Avg return in last 50 episodes: -200	 Episode return: -200	 Adjusted return:

Avg return in last 50 episodes: -199.98	 Episode return: -199	 Adjusted return: -187.5	 Total steps: 52199
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -200	 Total steps: 52399
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -200	 Total steps: 52599
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -189.5	 Total steps: 52799
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -200	 Total steps: 52999
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -172	 Total steps: 53199
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -200	 Total steps: 53399
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -188	 Total steps: 53599
Avg return in last 50 episodes: -199.98	 Episode return: -200	 Adjusted return: -190.5	 Total steps: 53799
Avg return in last 50 episodes: -199.98	 Episode 

# Final agent testing!

In [19]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "MountainCar-v0");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();

auto video = xw::video_from_url(url).finalize();
video

 Total steps: 200	 Total reward: -200


A Jupyter widget